# Environment
- GPU : NVIDIA GeForce GTX 1060
- CPU : Intel CORE i5 8th Gen

## Library version check

In [1]:
import sys
import tqdm as tq
import xgboost as xgb
import lightgbm as lgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("xgboost version: {}".format(xgb.__version__))
print("lightgbm version: {}".format(lgb.__version__))
print("seaborn version: {}".format(sns.__version__))
print("scikit-learn version: {}".format(skl.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
pandas version: 1.5.2
numpy version: 1.21.6
matplotlib version: 3.5.2
tqdm version: 4.65.2
xgboost version: 1.7.2
lightgbm version: 3.3.5
seaborn version: 0.11.2
scikit-learn version: 1.1.3
------------------------------------------------------------------------------


## 0. load the libararies

In [ ]:
import os
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tqdm import tqdm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings

pd.set_option('display.max_columns', 30)

## 1. Load Data

In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [ ]:
#BDI 외부데이터 load & preprocessing
bdi_data = pd.read_csv('./data/bdi.csv')

#예시 데이터프레임 생성 (2014년 8월 29일부터 2023년 2월 27일까지의 데이터)
start_date = datetime(2014, 8, 29)
end_date = datetime(2023, 2, 27)

#생성할 DataFrame 초기화
date_list = []
current_date = start_date

while current_date <= end_date:
    date_list.append(current_date)
    current_date += timedelta(days=1)

df = pd.DataFrame({'Date': date_list})

#인덱스 추가
df['Index'] = range(len(df))

#datetime 형식으로 바꿔주기 위해 문자열 재지정
bdi_data['Date'] = bdi_data['Date'].str.replace('/', '-')
bdi_data['Date'] = pd.to_datetime(bdi_data['Date'])

#dataframe columns rename
bdi_data.columns = ['Date','bdi_price','start','high','low','volume','moving']

#train, test에 merge 시킬 dataframe 생성
bdi_dataframe = pd.merge(df['Date'], bdi_data[['Date','bdi_price']], on='Date', how='outer').sort_values(by='Date')

#bdi 데이터에는 토,일 데이터가 존재하지 않으므로 금요일 값으로 대체
bdi_dataframe.fillna(method='ffill', inplace=True)

#bdi_price 데이터 타입변경
bdi_dataframe['bdi_price'] = [float(value.replace(',', '')) for value in bdi_dataframe['bdi_price']]

In [ ]:
#수입,수출 외부데이터 load & preprocessing
from dateutil.relativedelta import relativedelta

data = pd.read_csv('./data/최종데이터.csv')
data['Date'] = data['Date'] + '-01'
data['Date'] = pd.to_datetime(data['Date'])

#한달씩 미뤄주기(예를들어, 2015년 3월에 해당하는 데이터들을 ATA시점이 2015년 4월인 train,test 데이터에 병합시켜주기 위함)
data['merge_date'] = data['Date'] + data['Date'].apply(lambda x: relativedelta(months=1))
data['merge_date'] = pd.to_datetime(data['merge_date'])
data['year'] = data['merge_date'].dt.year
data['month'] = data['merge_date'].dt.month

## 2. EDA & Data Preprocessing

In [ ]:
#EDA전 datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday
    df['weekofyear'] = df['ATA'].dt.weekofyear

# datetime 컬럼 제거
train.drop(columns='ATA', inplace=True)
test.drop(columns='ATA', inplace=True)

In [ ]:
#BDI 외부데이터 병합
bdi_dataframe['Date'] = pd.to_datetime(bdi_dataframe['Date'])

#ATA기준 전날의 값으로 병합시켜주기 위해 전처리(수입,수출 데이터와 같은 맥락)
bdi_dataframe['merge_date'] = bdi_dataframe['Date'] + pd.to_timedelta(1, unit='d')

# datetime을 여러 파생 변수로 변환
bdi_dataframe['year'] = bdi_dataframe['merge_date'].dt.year
bdi_dataframe['month'] = bdi_dataframe['merge_date'].dt.month
bdi_dataframe['day'] = bdi_dataframe['merge_date'].dt.day

bdi_dataframe = bdi_dataframe.drop(columns =['merge_date'])

train = train.merge(bdi_dataframe, how = 'left', on = ['year','month','day'])
test = test.merge(bdi_dataframe, how = 'left', on = ['year','month','day'])

train.drop(columns=['Date'], inplace = True)
test.drop(columns=['Date'], inplace = True)

#예외 날짜에 대한 처리
train['bdi_price'] = train['bdi_price'].fillna(0)
test['bdi_price'] = test['bdi_price'].fillna(0)

display(train.head())

In [ ]:
#수입,수출 데이터 병합
data = data.drop(columns =['merge_date'])
data = data.drop(columns =['Date'])

train = train.merge(data, how = 'left', on = ['year','month'])
test = test.merge(data, how = 'left', on = ['year','month'])

#예외 날짜에 대한 처리
train.iloc[:,30:] = train.iloc[:,30:].fillna(0)
test.iloc[:,29:] = test.iloc[:,29:].fillna(0)

In [ ]:
#나라별 알맞은 데이터 할당
train.loc[(train['ARI_CO']=='TW'),'수출금액']=train['대만_수출금액']
train.loc[(train['ARI_CO']=='TW'),'수출증감률']=train['대만_수출증감률']
train.loc[(train['ARI_CO']=='TW'),'수입금액']=train['대만_수입금액']
train.loc[(train['ARI_CO']=='TW'),'수입증감률']=train['대만_수입증감률']
train.loc[(train['ARI_CO']=='TW'),'무역수지']=train['대만_무역수지']

train.loc[(train['ARI_CO']=='CN'),'수출금액']=train['중국_수출금액']
train.loc[(train['ARI_CO']=='CN'),'수출증감률']=train['중국_수출증감률']
train.loc[(train['ARI_CO']=='CN'),'수입금액']=train['중국_수입금액']
train.loc[(train['ARI_CO']=='CN'),'수입증감률']=train['중국_수입증감률']
train.loc[(train['ARI_CO']=='CN'),'무역수지']=train['중국_무역수지']

train.loc[(train['ARI_CO']=='JP'),'수출금액']=train['일본_수출금액']
train.loc[(train['ARI_CO']=='JP'),'수출증감률']=train['일본_수출증감률']
train.loc[(train['ARI_CO']=='JP'),'수입금액']=train['일본_수입금액']
train.loc[(train['ARI_CO']=='JP'),'수입증감률']=train['일본_수입증감률']
train.loc[(train['ARI_CO']=='JP'),'무역수지']=train['일본_무역수지']

train.loc[(train['ARI_CO']=='SG'),'수출금액']=train['싱가포르_수출금액']
train.loc[(train['ARI_CO']=='SG'),'수출증감률']=train['싱가포르_수출증감률']
train.loc[(train['ARI_CO']=='SG'),'수입금액']=train['싱가포르_수입금액']
train.loc[(train['ARI_CO']=='SG'),'수입증감률']=train['싱가포르_수입증감률']
train.loc[(train['ARI_CO']=='SG'),'무역수지']=train['싱가포르_무역수지']

train.loc[(train['ARI_CO']=='AU'),'수출금액']=train['호주_수출금액']
train.loc[(train['ARI_CO']=='AU'),'수출증감률']=train['호주_수출증감률']
train.loc[(train['ARI_CO']=='AU'),'수입금액']=train['호주_수입금액']
train.loc[(train['ARI_CO']=='AU'),'수입증감률']=train['호주_수입증감률']
train.loc[(train['ARI_CO']=='AU'),'무역수지']=train['호주_무역수지']

train.loc[(train['ARI_CO']=='IN'),'수출금액']=train['인도_수출금액']
train.loc[(train['ARI_CO']=='IN'),'수출증감률']=train['인도_수출증감률']
train.loc[(train['ARI_CO']=='IN'),'수입금액']=train['인도_수입금액']
train.loc[(train['ARI_CO']=='IN'),'수입증감률']=train['인도_수입증감률']
train.loc[(train['ARI_CO']=='IN'),'무역수지']=train['인도_무역수지']

train.loc[(train['ARI_CO']=='RU'),'수출금액']=train['러시아_수출금액']
train.loc[(train['ARI_CO']=='RU'),'수출증감률']=train['러시아_수출증감률']
train.loc[(train['ARI_CO']=='RU'),'수입금액']=train['러시아_수입금액']
train.loc[(train['ARI_CO']=='RU'),'수입증감률']=train['러시아_수입증감률']
train.loc[(train['ARI_CO']=='RU'),'무역수지']=train['러시아_무역수지']

train.loc[(train['ARI_CO']=='CA'),'수출금액']=train['캐나다_수출금액']
train.loc[(train['ARI_CO']=='CA'),'수출증감률']=train['캐나다_수출증감률']
train.loc[(train['ARI_CO']=='CA'),'수입금액']=train['캐나다_수입금액']
train.loc[(train['ARI_CO']=='CA'),'수입증감률']=train['캐나다_수입증감률']
train.loc[(train['ARI_CO']=='CA'),'무역수지']=train['캐나다_무역수지']

train.loc[(train['ARI_CO']=='KR'),'수출금액']=train['한국_수출금액']
train.loc[(train['ARI_CO']=='KR'),'수출증감률']=train['한국_수출증감률']
train.loc[(train['ARI_CO']=='KR'),'수입금액']=train['한국_수입금액']
train.loc[(train['ARI_CO']=='KR'),'수입증감률']=train['한국_수입증감률']
train.loc[(train['ARI_CO']=='KR'),'무역수지']=train['한국_무역수지']

train.loc[(train['ARI_CO']=='BR'),'수출금액']=train['브라질_수출금액']
train.loc[(train['ARI_CO']=='BR'),'수출증감률']=train['브라질_수출증감률']
train.loc[(train['ARI_CO']=='BR'),'수입금액']=train['브라질_수입금액']
train.loc[(train['ARI_CO']=='BR'),'수입증감률']=train['브라질_수입증감률']
train.loc[(train['ARI_CO']=='BR'),'무역수지']=train['브라질_무역수지']

train.loc[(train['ARI_CO']=='ID'),'수출금액']=train['인도네시아_수출금액']
train.loc[(train['ARI_CO']=='ID'),'수출증감률']=train['인도네시아_수출증감률']
train.loc[(train['ARI_CO']=='ID'),'수입금액']=train['인도네시아_수입금액']
train.loc[(train['ARI_CO']=='ID'),'수입증감률']=train['인도네시아_수입증감률']
train.loc[(train['ARI_CO']=='ID'),'무역수지']=train['인도네시아_무역수지']

train.loc[(train['ARI_CO']=='US'),'수출금액']=train['미국_수출금액']
train.loc[(train['ARI_CO']=='US'),'수출증감률']=train['미국_수출증감률']
train.loc[(train['ARI_CO']=='US'),'수입금액']=train['미국_수입금액']
train.loc[(train['ARI_CO']=='US'),'수입증감률']=train['미국_수입증감률']
train.loc[(train['ARI_CO']=='US'),'무역수지']=train['미국_무역수지']

train.loc[(train['ARI_CO']=='UA'),'수출금액']=train['우크라이나_수출금액']
train.loc[(train['ARI_CO']=='UA'),'수출증감률']=train['우크라이나_수출증감률']
train.loc[(train['ARI_CO']=='UA'),'수입금액']=train['우크라이나_수입금액']
train.loc[(train['ARI_CO']=='UA'),'수입증감률']=train['우크라이나_수입증감률']
train.loc[(train['ARI_CO']=='UA'),'무역수지']=train['우크라이나_무역수지']

train.loc[(train['ARI_CO']=='LV'),'수출금액']=train['라트비아_수출금액']
train.loc[(train['ARI_CO']=='LV'),'수출증감률']=train['라트비아_수출증감률']
train.loc[(train['ARI_CO']=='LV'),'수입금액']=train['라트비아_수입금액']
train.loc[(train['ARI_CO']=='LV'),'수입증감률']=train['라트비아_수입증감률']
train.loc[(train['ARI_CO']=='LV'),'무역수지']=train['라트비아_무역수지']

train.loc[(train['ARI_CO']=='MZ'),'수출금액']=train['모잠비크_수출금액']
train.loc[(train['ARI_CO']=='MZ'),'수출증감률']=train['모잠비크_수출증감률']
train.loc[(train['ARI_CO']=='MZ'),'수입금액']=train['모잠비크_수입금액']
train.loc[(train['ARI_CO']=='MZ'),'수입증감률']=train['모잠비크_수입증감률']
train.loc[(train['ARI_CO']=='MZ'),'무역수지']=train['모잠비크_무역수지']

train.loc[(train['ARI_CO']=='QA'),'수출금액']=train['카타르_수출금액']
train.loc[(train['ARI_CO']=='QA'),'수출증감률']=train['카타르_수출증감률']
train.loc[(train['ARI_CO']=='QA'),'수입금액']=train['카타르_수입금액']
train.loc[(train['ARI_CO']=='QA'),'수입증감률']=train['카타르_수입증감률']
train.loc[(train['ARI_CO']=='QA'),'무역수지']=train['카타르_무역수지']

train.loc[(train['ARI_CO']=='ZA'),'수출금액']=train['남아프리카공화국_수출금액']
train.loc[(train['ARI_CO']=='ZA'),'수출증감률']=train['남아프리카공화국_수출증감률']
train.loc[(train['ARI_CO']=='ZA'),'수입금액']=train['남아프리카공화국_수입금액']
train.loc[(train['ARI_CO']=='ZA'),'수입증감률']=train['남아프리카공화국_수입증감률']
train.loc[(train['ARI_CO']=='ZA'),'무역수지']=train['남아프리카공화국_무역수지']

train.loc[(train['ARI_CO']=='VN'),'수출금액']=train['베트남_수출금액']
train.loc[(train['ARI_CO']=='VN'),'수출증감률']=train['베트남_수출증감률']
train.loc[(train['ARI_CO']=='VN'),'수입금액']=train['베트남_수입금액']
train.loc[(train['ARI_CO']=='VN'),'수입증감률']=train['베트남_수입증감률']
train.loc[(train['ARI_CO']=='VN'),'무역수지']=train['베트남_무역수지']

train.loc[(train['ARI_CO']=='TT'),'수출금액']=train['트리니다드토바고_수출금액']
train.loc[(train['ARI_CO']=='TT'),'수출증감률']=train['트리니다드토바고_수출증감률']
train.loc[(train['ARI_CO']=='TT'),'수입금액']=train['트리니다드토바고_수입금액']
train.loc[(train['ARI_CO']=='TT'),'수입증감률']=train['트리니다드토바고_수입증감률']
train.loc[(train['ARI_CO']=='TT'),'무역수지']=train['트리니다드토바고_무역수지']

train.loc[(train['ARI_CO']=='PE'),'수출금액']=train['페루_수출금액']
train.loc[(train['ARI_CO']=='PE'),'수출증감률']=train['페루_수출증감률']
train.loc[(train['ARI_CO']=='PE'),'수입금액']=train['페루_수입금액']
train.loc[(train['ARI_CO']=='PE'),'수입증감률']=train['페루_수입증감률']
train.loc[(train['ARI_CO']=='PE'),'무역수지']=train['페루_무역수지']

train.loc[(train['ARI_CO']=='MY'),'수출금액']=train['말레이시아_수출금액']
train.loc[(train['ARI_CO']=='MY'),'수출증감률']=train['말레이시아_수출증감률']
train.loc[(train['ARI_CO']=='MY'),'수입금액']=train['말레이시아_수입금액']
train.loc[(train['ARI_CO']=='MY'),'수입증감률']=train['말레이시아_수입증감률']
train.loc[(train['ARI_CO']=='MY'),'무역수지']=train['말레이시아_무역수지']

train.loc[(train['ARI_CO']=='CL'),'수출금액']=train['칠레_수출금액']
train.loc[(train['ARI_CO']=='CL'),'수출증감률']=train['칠레_수출증감률']
train.loc[(train['ARI_CO']=='CL'),'수입금액']=train['칠레_수입금액']
train.loc[(train['ARI_CO']=='CL'),'수입증감률']=train['칠레_수입증감률']
train.loc[(train['ARI_CO']=='CL'),'무역수지']=train['칠레_무역수지']

train.loc[(train['ARI_CO']=='FI'),'수출금액']=train['핀란드_수출금액']
train.loc[(train['ARI_CO']=='FI'),'수출증감률']=train['핀란드_수출증감률']
train.loc[(train['ARI_CO']=='FI'),'수입금액']=train['핀란드_수입금액']
train.loc[(train['ARI_CO']=='FI'),'수입증감률']=train['핀란드_수입증감률']
train.loc[(train['ARI_CO']=='FI'),'무역수지']=train['핀란드_무역수지']

train.loc[(train['ARI_CO']=='PH'),'수출금액']=train['필리핀_수출금액']
train.loc[(train['ARI_CO']=='PH'),'수출증감률']=train['필리핀_수출증감률']
train.loc[(train['ARI_CO']=='PH'),'수입금액']=train['필리핀_수입금액']
train.loc[(train['ARI_CO']=='PH'),'수입증감률']=train['필리핀_수입증감률']
train.loc[(train['ARI_CO']=='PH'),'무역수지']=train['필리핀_무역수지']

#예외 데이터 처리(Null 값 처리)
train['수출금액'] = train['수출금액'].fillna(train['미국_수출금액'])
train['수출증감률'] = train['수출증감률'].fillna(train['미국_수출증감률'])
train['수입금액'] = train['수입금액'].fillna(train['미국_수입금액'])
train['수입증감률'] = train['수입증감률'].fillna(train['미국_수입증감률'])
train['무역수지'] = train['무역수지'].fillna(train['미국_무역수지'])

In [ ]:
test.loc[(test['ARI_CO']=='TW'),'수출금액']=test['대만_수출금액']
test.loc[(test['ARI_CO']=='TW'),'수출증감률']=test['대만_수출증감률']
test.loc[(test['ARI_CO']=='TW'),'수입금액']=test['대만_수입금액']
test.loc[(test['ARI_CO']=='TW'),'수입증감률']=test['대만_수입증감률']
test.loc[(test['ARI_CO']=='TW'),'무역수지']=test['대만_무역수지']

test.loc[(test['ARI_CO']=='CN'),'수출금액']=test['중국_수출금액']
test.loc[(test['ARI_CO']=='CN'),'수출증감률']=test['중국_수출증감률']
test.loc[(test['ARI_CO']=='CN'),'수입금액']=test['중국_수입금액']
test.loc[(test['ARI_CO']=='CN'),'수입증감률']=test['중국_수입증감률']
test.loc[(test['ARI_CO']=='CN'),'무역수지']=test['중국_무역수지']

test.loc[(test['ARI_CO']=='JP'),'수출금액']=test['일본_수출금액']
test.loc[(test['ARI_CO']=='JP'),'수출증감률']=test['일본_수출증감률']
test.loc[(test['ARI_CO']=='JP'),'수입금액']=test['일본_수입금액']
test.loc[(test['ARI_CO']=='JP'),'수입증감률']=test['일본_수입증감률']
test.loc[(test['ARI_CO']=='JP'),'무역수지']=test['일본_무역수지']

test.loc[(test['ARI_CO']=='SG'),'수출금액']=test['싱가포르_수출금액']
test.loc[(test['ARI_CO']=='SG'),'수출증감률']=test['싱가포르_수출증감률']
test.loc[(test['ARI_CO']=='SG'),'수입금액']=test['싱가포르_수입금액']
test.loc[(test['ARI_CO']=='SG'),'수입증감률']=test['싱가포르_수입증감률']
test.loc[(test['ARI_CO']=='SG'),'무역수지']=test['싱가포르_무역수지']

test.loc[(test['ARI_CO']=='AU'),'수출금액']=test['호주_수출금액']
test.loc[(test['ARI_CO']=='AU'),'수출증감률']=test['호주_수출증감률']
test.loc[(test['ARI_CO']=='AU'),'수입금액']=test['호주_수입금액']
test.loc[(test['ARI_CO']=='AU'),'수입증감률']=test['호주_수입증감률']
test.loc[(test['ARI_CO']=='AU'),'무역수지']=test['호주_무역수지']

test.loc[(test['ARI_CO']=='IN'),'수출금액']=test['인도_수출금액']
test.loc[(test['ARI_CO']=='IN'),'수출증감률']=test['인도_수출증감률']
test.loc[(test['ARI_CO']=='IN'),'수입금액']=test['인도_수입금액']
test.loc[(test['ARI_CO']=='IN'),'수입증감률']=test['인도_수입증감률']
test.loc[(test['ARI_CO']=='IN'),'무역수지']=test['인도_무역수지']

test.loc[(test['ARI_CO']=='RU'),'수출금액']=test['러시아_수출금액']
test.loc[(test['ARI_CO']=='RU'),'수출증감률']=test['러시아_수출증감률']
test.loc[(test['ARI_CO']=='RU'),'수입금액']=test['러시아_수입금액']
test.loc[(test['ARI_CO']=='RU'),'수입증감률']=test['러시아_수입증감률']
test.loc[(test['ARI_CO']=='RU'),'무역수지']=test['러시아_무역수지']

test.loc[(test['ARI_CO']=='CA'),'수출금액']=test['캐나다_수출금액']
test.loc[(test['ARI_CO']=='CA'),'수출증감률']=test['캐나다_수출증감률']
test.loc[(test['ARI_CO']=='CA'),'수입금액']=test['캐나다_수입금액']
test.loc[(test['ARI_CO']=='CA'),'수입증감률']=test['캐나다_수입증감률']
test.loc[(test['ARI_CO']=='CA'),'무역수지']=test['캐나다_무역수지']

test.loc[(test['ARI_CO']=='KR'),'수출금액']=test['한국_수출금액']
test.loc[(test['ARI_CO']=='KR'),'수출증감률']=test['한국_수출증감률']
test.loc[(test['ARI_CO']=='KR'),'수입금액']=test['한국_수입금액']
test.loc[(test['ARI_CO']=='KR'),'수입증감률']=test['한국_수입증감률']
test.loc[(test['ARI_CO']=='KR'),'무역수지']=test['한국_무역수지']

test.loc[(test['ARI_CO']=='BR'),'수출금액']=test['브라질_수출금액']
test.loc[(test['ARI_CO']=='BR'),'수출증감률']=test['브라질_수출증감률']
test.loc[(test['ARI_CO']=='BR'),'수입금액']=test['브라질_수입금액']
test.loc[(test['ARI_CO']=='BR'),'수입증감률']=test['브라질_수입증감률']
test.loc[(test['ARI_CO']=='BR'),'무역수지']=test['브라질_무역수지']

test.loc[(test['ARI_CO']=='ID'),'수출금액']=test['인도네시아_수출금액']
test.loc[(test['ARI_CO']=='ID'),'수출증감률']=test['인도네시아_수출증감률']
test.loc[(test['ARI_CO']=='ID'),'수입금액']=test['인도네시아_수입금액']
test.loc[(test['ARI_CO']=='ID'),'수입증감률']=test['인도네시아_수입증감률']
test.loc[(test['ARI_CO']=='ID'),'무역수지']=test['인도네시아_무역수지']

test.loc[(test['ARI_CO']=='US'),'수출금액']=test['미국_수출금액']
test.loc[(test['ARI_CO']=='US'),'수출증감률']=test['미국_수출증감률']
test.loc[(test['ARI_CO']=='US'),'수입금액']=test['미국_수입금액']
test.loc[(test['ARI_CO']=='US'),'수입증감률']=test['미국_수입증감률']
test.loc[(test['ARI_CO']=='US'),'무역수지']=test['미국_무역수지']

test.loc[(test['ARI_CO']=='UA'),'수출금액']=test['우크라이나_수출금액']
test.loc[(test['ARI_CO']=='UA'),'수출증감률']=test['우크라이나_수출증감률']
test.loc[(test['ARI_CO']=='UA'),'수입금액']=test['우크라이나_수입금액']
test.loc[(test['ARI_CO']=='UA'),'수입증감률']=test['우크라이나_수입증감률']
test.loc[(test['ARI_CO']=='UA'),'무역수지']=test['우크라이나_무역수지']

test.loc[(test['ARI_CO']=='LV'),'수출금액']=test['라트비아_수출금액']
test.loc[(test['ARI_CO']=='LV'),'수출증감률']=test['라트비아_수출증감률']
test.loc[(test['ARI_CO']=='LV'),'수입금액']=test['라트비아_수입금액']
test.loc[(test['ARI_CO']=='LV'),'수입증감률']=test['라트비아_수입증감률']
test.loc[(test['ARI_CO']=='LV'),'무역수지']=test['라트비아_무역수지']

test.loc[(test['ARI_CO']=='MZ'),'수출금액']=test['모잠비크_수출금액']
test.loc[(test['ARI_CO']=='MZ'),'수출증감률']=test['모잠비크_수출증감률']
test.loc[(test['ARI_CO']=='MZ'),'수입금액']=test['모잠비크_수입금액']
test.loc[(test['ARI_CO']=='MZ'),'수입증감률']=test['모잠비크_수입증감률']
test.loc[(test['ARI_CO']=='MZ'),'무역수지']=test['모잠비크_무역수지']

test.loc[(test['ARI_CO']=='QA'),'수출금액']=test['카타르_수출금액']
test.loc[(test['ARI_CO']=='QA'),'수출증감률']=test['카타르_수출증감률']
test.loc[(test['ARI_CO']=='QA'),'수입금액']=test['카타르_수입금액']
test.loc[(test['ARI_CO']=='QA'),'수입증감률']=test['카타르_수입증감률']
test.loc[(test['ARI_CO']=='QA'),'무역수지']=test['카타르_무역수지']

test.loc[(test['ARI_CO']=='ZA'),'수출금액']=test['남아프리카공화국_수출금액']
test.loc[(test['ARI_CO']=='ZA'),'수출증감률']=test['남아프리카공화국_수출증감률']
test.loc[(test['ARI_CO']=='ZA'),'수입금액']=test['남아프리카공화국_수입금액']
test.loc[(test['ARI_CO']=='ZA'),'수입증감률']=test['남아프리카공화국_수입증감률']
test.loc[(test['ARI_CO']=='ZA'),'무역수지']=test['남아프리카공화국_무역수지']

test.loc[(test['ARI_CO']=='VN'),'수출금액']=test['베트남_수출금액']
test.loc[(test['ARI_CO']=='VN'),'수출증감률']=test['베트남_수출증감률']
test.loc[(test['ARI_CO']=='VN'),'수입금액']=test['베트남_수입금액']
test.loc[(test['ARI_CO']=='VN'),'수입증감률']=test['베트남_수입증감률']
test.loc[(test['ARI_CO']=='VN'),'무역수지']=test['베트남_무역수지']

test.loc[(test['ARI_CO']=='TT'),'수출금액']=test['트리니다드토바고_수출금액']
test.loc[(test['ARI_CO']=='TT'),'수출증감률']=test['트리니다드토바고_수출증감률']
test.loc[(test['ARI_CO']=='TT'),'수입금액']=test['트리니다드토바고_수입금액']
test.loc[(test['ARI_CO']=='TT'),'수입증감률']=test['트리니다드토바고_수입증감률']
test.loc[(test['ARI_CO']=='TT'),'무역수지']=test['트리니다드토바고_무역수지']

test.loc[(test['ARI_CO']=='PE'),'수출금액']=test['페루_수출금액']
test.loc[(test['ARI_CO']=='PE'),'수출증감률']=test['페루_수출증감률']
test.loc[(test['ARI_CO']=='PE'),'수입금액']=test['페루_수입금액']
test.loc[(test['ARI_CO']=='PE'),'수입증감률']=test['페루_수입증감률']
test.loc[(test['ARI_CO']=='PE'),'무역수지']=test['페루_무역수지']

test.loc[(test['ARI_CO']=='MY'),'수출금액']=test['말레이시아_수출금액']
test.loc[(test['ARI_CO']=='MY'),'수출증감률']=test['말레이시아_수출증감률']
test.loc[(test['ARI_CO']=='MY'),'수입금액']=test['말레이시아_수입금액']
test.loc[(test['ARI_CO']=='MY'),'수입증감률']=test['말레이시아_수입증감률']
test.loc[(test['ARI_CO']=='MY'),'무역수지']=test['말레이시아_무역수지']

test.loc[(test['ARI_CO']=='CL'),'수출금액']=test['칠레_수출금액']
test.loc[(test['ARI_CO']=='CL'),'수출증감률']=test['칠레_수출증감률']
test.loc[(test['ARI_CO']=='CL'),'수입금액']=test['칠레_수입금액']
test.loc[(test['ARI_CO']=='CL'),'수입증감률']=test['칠레_수입증감률']
test.loc[(test['ARI_CO']=='CL'),'무역수지']=test['칠레_무역수지']

test.loc[(test['ARI_CO']=='FI'),'수출금액']=test['핀란드_수출금액']
test.loc[(test['ARI_CO']=='FI'),'수출증감률']=test['핀란드_수출증감률']
test.loc[(test['ARI_CO']=='FI'),'수입금액']=test['핀란드_수입금액']
test.loc[(test['ARI_CO']=='FI'),'수입증감률']=test['핀란드_수입증감률']
test.loc[(test['ARI_CO']=='FI'),'무역수지']=test['핀란드_무역수지']

test.loc[(test['ARI_CO']=='PH'),'수출금액']=test['필리핀_수출금액']
test.loc[(test['ARI_CO']=='PH'),'수출증감률']=test['필리핀_수출증감률']
test.loc[(test['ARI_CO']=='PH'),'수입금액']=test['필리핀_수입금액']
test.loc[(test['ARI_CO']=='PH'),'수입증감률']=test['필리핀_수입증감률']
test.loc[(test['ARI_CO']=='PH'),'무역수지']=test['필리핀_무역수지']

test['수출금액'] = test['수출금액'].fillna(test['미국_수출금액'])
test['수출증감률'] = test['수출증감률'].fillna(test['미국_수출증감률'])
test['수입금액'] = test['수입금액'].fillna(test['미국_수입금액'])
test['수입증감률'] = test['수입증감률'].fillna(test['미국_수입증감률'])
test['무역수지'] = test['무역수지'].fillna(test['미국_무역수지'])

In [ ]:
#수출금액,수출증감률,수입금액,수입증감률,무역수지 columns만 사용
columns_list_tr = train.columns[:30].to_list() + train.columns[-5:].to_list()
train = train[columns_list_tr]
columns_list_te = train.drop(columns=['CI_HOUR']).columns.to_list()
test = test[columns_list_te]

### 2-1. Target Analysis

In [ ]:
#시각화 설정
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 15
# 사용자 운영체제 확인
os.name

# 운영체제별 한글 폰트 설정
if os.name == 'posix': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif os.name == 'nt': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정

# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

In [ ]:
#Target distribuition
plt.figure(figsize=(10,10))
x = train['CI_HOUR']
sns.kdeplot(x)
plt.title("Target")
plt.show()

In [ ]:
train['CI_HOUR'].describe()
#매우 skew가 높은 분포이므로 후에 log변환등 정규화 과정이 필요할 수 있다고 판단

In [ ]:
#Target distribuition(not zero)
plt.figure(figsize=(10,10))
x = train[train['CI_HOUR'] != 0]['CI_HOUR']
sns.kdeplot(x)
plt.title("Target")
plt.show()

In [ ]:
train[train['CI_HOUR'] != 0]['CI_HOUR'].describe()
#0 제외한 값에 대해서도 표준편차가 매우 크고 skew가 큰 분포

In [ ]:
train['CI_HOUR'].value_counts()
#0이 매우 많은 분포

In [ ]:
#DIST가 0인 경우 Target 값이 대부분 0이므로 그 외의 데이터들은 이상치로 판단 > Target값 0으로 대체
train.loc[(train['DIST'] == 0) & (train['CI_HOUR'] > 0), 'CI_HOUR'] = 0.0

### 2-2. NaN

In [ ]:
#BREADTH, DEPTH, DRAUGHT, LENGTH NaN Row
train[train['BREADTH'].isna()]

In [ ]:
#선박의 DEADWEIGHT(재화중량톤수), GT(용적톤수) 및 FLAG(국적)에 따라 BREADTH, DEPTH, DRAUGHT, LENGTH 결측치 대체
train[(train['DEADWEIGHT'] == 1500) & (train['GT'] == 500)]

In [ ]:
train.loc[train['BREADTH'].isna(), 'BREADTH'] = 10.0
train.loc[train['DEPTH'].isna(), 'DEPTH'] = 10.0
train.loc[train['DRAUGHT'].isna(), 'DRAUGHT'] = 0.0
train.loc[train['LENGTH'].isna(), 'LENGTH'] = 70.0

In [ ]:
#CGT
con_cargo = ['Container','Cargo']
train.loc[(train['DEADWEIGHT']>=250000)  & (train['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=train['DEADWEIGHT'] * 0.25
train.loc[(train['DEADWEIGHT']>=160000) & (train['DEADWEIGHT']<250000) & (train['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=train['DEADWEIGHT'] * 0.3
train.loc[(train['DEADWEIGHT']>=80000) & (train['DEADWEIGHT']<160000) & (train['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=train['DEADWEIGHT'] * 0.4
train.loc[(train['DEADWEIGHT']>=50000) & (train['DEADWEIGHT']<80000) & (train['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=train['DEADWEIGHT'] * 0.5
train.loc[(train['DEADWEIGHT']>=30000) & (train['DEADWEIGHT']<50000) & (train['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=train['DEADWEIGHT'] * 0.65
train.loc[(train['DEADWEIGHT']>=10000) & (train['DEADWEIGHT']<30000) & (train['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=train['DEADWEIGHT'] * 0.75
train.loc[(train['DEADWEIGHT']>=4000) & (train['DEADWEIGHT']<10000) & (train['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=train['DEADWEIGHT'] * 1.15
train.loc[(train['DEADWEIGHT']<4000)  & (train['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=train['DEADWEIGHT'] * 1.7

train.loc[(train['DEADWEIGHT']>=160000)  & (train['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=train['DEADWEIGHT'] * 0.3
train.loc[(train['DEADWEIGHT']>=80000) & (train['DEADWEIGHT']<160000) & (train['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=train['DEADWEIGHT'] * 0.4
train.loc[(train['DEADWEIGHT']>=50000) & (train['DEADWEIGHT']<80000) & (train['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=train['DEADWEIGHT'] * 0.5
train.loc[(train['DEADWEIGHT']>=30000) & (train['DEADWEIGHT']<50000) & (train['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=train['DEADWEIGHT'] * 0.6
train.loc[(train['DEADWEIGHT']>=10000) & (train['DEADWEIGHT']<30000) & (train['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=train['DEADWEIGHT'] * 0.7
train.loc[(train['DEADWEIGHT']>=4000) & (train['DEADWEIGHT']<10000) & (train['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=train['DEADWEIGHT'] * 1.1
train.loc[(train['DEADWEIGHT']<4000)  & (train['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=train['DEADWEIGHT'] * 1.6

train.loc[(train['DEADWEIGHT']>=10000)  & (train['SHIP_TYPE_CATEGORY'].isin(con_cargo)),'CGT']=train['DEADWEIGHT'] * 1.25
train.loc[(train['DEADWEIGHT']>=4000) & (train['DEADWEIGHT']<10000) & (train['SHIP_TYPE_CATEGORY'].isin(con_cargo)),'CGT']=train['DEADWEIGHT'] * 1.5
train.loc[(train['DEADWEIGHT']<4000)  & (train['SHIP_TYPE_CATEGORY'].isin(con_cargo)),'CGT']=train['DEADWEIGHT'] * 2.05

#위에 해당하지 않는 값에 대한 처리(해당 Row의 DEADWEIGHT값으로 대체)
train['CGT'].fillna(train['DEADWEIGHT'], inplace=True)

test.loc[(test['DEADWEIGHT']>=250000)  & (test['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=test['DEADWEIGHT'] * 0.25
test.loc[(test['DEADWEIGHT']>=160000) & (test['DEADWEIGHT']<250000) & (test['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=test['DEADWEIGHT'] * 0.3
test.loc[(test['DEADWEIGHT']>=80000) & (test['DEADWEIGHT']<160000) & (test['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=test['DEADWEIGHT'] * 0.4
test.loc[(test['DEADWEIGHT']>=50000) & (test['DEADWEIGHT']<80000) & (test['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=test['DEADWEIGHT'] * 0.5
test.loc[(test['DEADWEIGHT']>=30000) & (test['DEADWEIGHT']<50000) & (test['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=test['DEADWEIGHT'] * 0.65
test.loc[(test['DEADWEIGHT']>=10000) & (test['DEADWEIGHT']<30000) & (test['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=test['DEADWEIGHT'] * 0.75
test.loc[(test['DEADWEIGHT']>=4000) & (test['DEADWEIGHT']<10000) & (test['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=test['DEADWEIGHT'] * 1.15
test.loc[(test['DEADWEIGHT']<4000)  & (test['SHIP_TYPE_CATEGORY']=='Tanker'),'CGT']=test['DEADWEIGHT'] * 1.7


test.loc[(test['DEADWEIGHT']>=160000)  & (test['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=test['DEADWEIGHT'] * 0.3
test.loc[(test['DEADWEIGHT']>=80000) & (test['DEADWEIGHT']<160000) & (test['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=test['DEADWEIGHT'] * 0.4
test.loc[(test['DEADWEIGHT']>=50000) & (test['DEADWEIGHT']<80000) & (test['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=test['DEADWEIGHT'] * 0.5
test.loc[(test['DEADWEIGHT']>=30000) & (test['DEADWEIGHT']<50000) & (test['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=test['DEADWEIGHT'] * 0.6
test.loc[(test['DEADWEIGHT']>=10000) & (test['DEADWEIGHT']<30000) & (test['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=test['DEADWEIGHT'] * 0.7
test.loc[(test['DEADWEIGHT']>=4000) & (test['DEADWEIGHT']<10000) & (test['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=test['DEADWEIGHT'] * 1.1
test.loc[(test['DEADWEIGHT']<4000)  & (test['SHIP_TYPE_CATEGORY']=='Bulk'),'CGT']=test['DEADWEIGHT'] * 1.6


test.loc[(test['DEADWEIGHT']>=10000)  & (test['SHIP_TYPE_CATEGORY'].isin(con_cargo)),'CGT']=test['DEADWEIGHT'] * 1.25
test.loc[(test['DEADWEIGHT']>=4000) & (test['DEADWEIGHT']<10000) & (test['SHIP_TYPE_CATEGORY'].isin(con_cargo)),'CGT']=test['DEADWEIGHT'] * 1.5
test.loc[(test['DEADWEIGHT']<4000)  & (test['SHIP_TYPE_CATEGORY'].isin(con_cargo)),'CGT']=test['DEADWEIGHT'] * 2.05

test['CGT'].fillna(test['DEADWEIGHT'], inplace=True)

In [ ]:
#나라_항구 컬럼 생성
train['ARI_INFO'] = train['ARI_CO'] + '_' + train['ARI_PO']
test['ARI_INFO'] = test['ARI_CO'] + '_' + test['ARI_PO']

## 3. Feature Engineering

In [ ]:
#DIST가 0인 데이터들을 함께 학습시, 학습이 잘 안되는 점을 보완하기 위해 DIST가 0이 아닌 데이터들만 사용하여 학습 진행
train['DIST'] = train['DIST'].fillna(0)
test['DIST'] = test['DIST'].fillna(0)

train = train.loc[(train['DIST'] != 0), ].reset_index(drop=True)

In [ ]:
#나라별 공휴일
import holidays

##나라별 공휴일 변수지정(TT,QA는 holidays 라이브러리로 처리 불가능하므로 따로 진행)
CN_holidays = holidays.CN()
JP_holidays = holidays.JP()
RU_holidays = holidays.RU()
AU_holidays = holidays.AU()
SG_holidays = holidays.SG()
ZA_holidays = holidays.ZA()
KR_holidays = holidays.KR()
TW_holidays = holidays.TW()
#TT_holidays = holidays.TT()
ID_holidays = holidays.ID()
BR_holidays = holidays.BR()
#QA_holidays = holidays.QA()
LV_holidays = holidays.LV()
MZ_holidays = holidays.MZ()
US_holidays = holidays.US()
IN_holidays = holidays.IN()
UA_holidays = holidays.UA()
CA_holidays = holidays.CA()
MY_holidays = holidays.MY()
PE_holidays = holidays.PE()
VN_holidays = holidays.VN()
FI_holidays = holidays.FI()
CL_holidays = holidays.CL()
VE_holidays = holidays.VE()
PH_holidays = holidays.PH()

##나라코드 리스트 생성
country_list = list(train['ARI_CO'].unique())
country_list.remove('TT')
country_list.remove('QA')

##나라별 공휴일 리스트 생성
holiday_list = [CN_holidays,JP_holidays,RU_holidays,AU_holidays,SG_holidays,
                ZA_holidays,KR_holidays,TW_holidays,ID_holidays,BR_holidays,
                LV_holidays,MZ_holidays,US_holidays,IN_holidays,UA_holidays,
                CA_holidays,MY_holidays,PE_holidays,VN_holidays,FI_holidays,
                CL_holidays,VE_holidays,PH_holidays]

##공휴일 피처생성
train['date'] = train['year'].astype(str) + '-' + train['month'].astype(str) + '-' + train['day'].astype(str)
test['date'] = test['year'].astype(str) + '-' + test['month'].astype(str) + '-' + test['day'].astype(str)

train['holiday'] = 0
test['holiday'] = 0
for i in tqdm(range(len(holiday_list))):
    train.loc[train['ARI_CO'] == country_list[i], "holiday"] = train.loc[train['ARI_CO'] == country_list[i], "date"].apply(lambda x : 1 if x in holiday_list[i] else 0)
    test.loc[test['ARI_CO'] == country_list[i], "holiday"] = test.loc[test['ARI_CO'] == country_list[i], "date"].apply(lambda x : 1 if x in holiday_list[i] else 0)
    
###트리니다드 토바고(TT) - 구글 Bard를 이용해 얻은 도메인 지식을 활용
TT_holiday = ['2014-11-1','2014-12-25','2014-12-26',
              '2015-1-1','2015-1-2','2015-1-25','2015-1-26','2015-1-27','2015-2-27','2015-3-8','2015-4-3','2015-4-6','2015-5-1','2015-5-22','2015-6-18','2015-8-3','2015-8-4','2015-11-1','2015-12-25','2015-12-26',
             '2016-1-1','2016-1-2','2016-1-24','2016-1-25','2016-1-26','2016-2-26','2016-3-8','2016-3-25','2016-3-28','2016-5-1','2016-5-22','2016-6-17','2016-8-3','2016-8-4','2016-11-1','2016-12-25','2016-12-26',
             '2017-1-1','2017-1-2','2017-1-23','2017-1-24','2017-1-25','2017-2-25','2017-3-8','2017-4-14','2017-4-17','2017-5-1','2017-5-22','2017-6-16','2017-8-3','2017-8-4','2017-11-1','2017-12-25','2017-12-26',
             '2018-1-1','2018-1-2','2018-1-22','2018-1-23','2018-1-24','2018-2-24','2018-3-8','2018-4-13','2018-4-16','2018-5-1','2018-5-22','2018-6-15','2018-8-3','2018-8-4','2018-11-1','2018-12-25','2018-12-26',
             '2019-1-1','2019-1-2','2019-1-21','2019-1-22','2019-1-23','2019-2-23','2019-3-8','2019-4-19','2019-4-22','2019-5-1','2019-5-22','2019-6-14','2019-8-3','2019-8-4','2019-11-1','2019-12-25','2019-12-26',
             '2020-1-1','2020-1-2','2020-1-20','2020-1-21','2020-1-22','2020-2-22','2020-3-8','2020-4-10','2020-4-13','2020-5-1','2020-5-22','2020-6-13','2020-8-3','2020-8-4','2020-11-1','2020-12-25','2020-12-26',
             '2021-1-1','2021-1-2','2021-1-19','2021-1-20','2021-1-21','2021-2-21','2021-3-8','2021-4-2','2021-4-5','2021-5-1','2021-5-22','2021-6-12','2021-8-3','2021-8-4','2021-11-1','2021-12-25','2021-12-26',
             '2022-1-1','2022-1-2','2022-1-18','2022-1-19','2022-1-20','2022-2-20','2022-3-8','2022-4-7','2022-4-10','2022-5-1','2022-5-22','2022-6-11','2022-8-3','2022-8-4','2022-11-1','2022-12-25','2022-12-26',
             '2023-1-1','2023-1-2','2023-1-17','2023-1-18','2023-1-19','2023-2-19']
train.loc[train['ARI_CO'] == "TT", "holiday"] = train.loc[train['ARI_CO'] == "TT", "date"].apply(lambda x : 1 if x in TT_holiday else 0)  
test.loc[test['ARI_CO'] == "TT", "holiday"] = test.loc[test['ARI_CO'] == "TT", "date"].apply(lambda x : 1 if x in TT_holiday else 0)  

###카타르(QA) - 구글 Bard를 이용해 얻은 도메인 지식을 활용
QA_holiday = ['2014-9-22','2014-9-23','2014-9-24','2014-12-25','2014-12-26',
             '2015-1-1','2015-1-2','2015-7-17','2015-7-18','2015-7-19','2015-9-21','2015-9-22','2015-9-23','2015-12-25','2015-12-26',
             '2016-1-1','2016-1-2','2016-7-10','2016-7-11','2016-7-12','2016-9-24','2016-9-25','2016-9-26','2016-12-25','2016-12-26',
             '2017-1-1','2017-1-2','2017-7-7','2017-7-8','2017-7-9','2017-9-22','2017-9-23','2017-9-24','2017-12-25','2017-12-26',
             '2018-1-1','2018-1-2','2018-7-14','2018-7-15','2018-7-16','2018-9-29','2018-9-30','2018-10-1','2018-12-25','2018-12-26',
             '2019-1-1','2019-1-2','2019-7-12','2019-7-13','2019-7-14','2019-9-27','2019-9-28','2019-9-29','2019-12-25','2019-12-26',
             '2020-1-1','2020-1-2','2020-5-27','2020-6-23','2020-6-24','2020-6-25','2020-7-31','2020-8-1','2020-8-2','2020-12-25','2020-12-26',
             '2021-1-1','2021-1-2','2021-5-27','2021-7-20','2021-7-21','2021-7-22','2021-9-10','2021-9-11','2021-9-12','2021-12-25','2021-12-26',
             '2022-1-1','2022-1-2','2022-5-27','2022-7-10','2022-7-11','2022-7-12','2022-9-24','2022-9-25','2022-9-26','2022-12-25','2022-12-26',
             '2023-1-1','2023-1-2']
train.loc[train['ARI_CO'] == "QA", "holiday"] = train.loc[train['ARI_CO'] == "QA", "date"].apply(lambda x : 1 if x in QA_holiday else 0)  
test.loc[test['ARI_CO'] == "QA", "holiday"] = test.loc[test['ARI_CO'] == "QA", "date"].apply(lambda x : 1 if x in QA_holiday else 0)

#Null 값 처리(다른 나라 및 날짜에 대한 처리)
train['holiday'] = train['holiday'].fillna(0)
test['holiday'] = test['holiday'].fillna(0)

In [ ]:
def group_covid(df):
    df['group_covid'] = '-' 
    df.loc[(df['year'] <= 2019), 'group_covid'] = 'prior_covid'
    df.loc[(df['year'] >= 2020) & (df['year'] < 2023), 'group_covid'] = 'pro_covid'
    df.loc[(df['year'] >= 2023), 'group_covid'] = 'after_covid'
    df.loc[(df['group_covid'] == '-'), 'group_covid'] = 'Na'
    return df['group_covid']

def group_season(df):
    df['group_season'] = '-'
    df.loc[(df['month'] == 3) | (df['month'] == 4) | (df['month'] == 5), 'group_season'] = 'season_1'
    df.loc[(df['month'] == 6) | (df['month'] == 7) | (df['month'] == 8), 'group_season'] = 'season_2'
    df.loc[(df['month'] == 9) | (df['month'] == 10) | (df['month'] == 11), 'group_season'] = 'season_3'
    df.loc[(df['month'] == 12) | (df['month'] == 1) | (df['month'] == 2), 'group_season'] = 'season_4'
    df.loc[(df['group_season'] == '-'), 'group_season'] = 'Na'
    return df['group_season']

def group_day(df):
    df['group_day'] = '-' 
    df.loc[(df['day'] < 10), 'group_day'] = 'start_month'
    df.loc[(df['day'] >= 10) & (df['day'] < 21), 'group_day'] = 'middle_month'
    df.loc[(df['day'] >= 21) & (df['day'] <= 31), 'group_day'] = 'finish_month'
    df.loc[(df['group_day'] == '-'), 'group_day'] = 'Na'
    return df['group_day']

def group_time(df):
    df['group_time'] = '-' 
    df.loc[(df['ATA_LT'] < 5), 'group_time'] = 'dawn'
    df.loc[(df['ATA_LT'] >= 5) & (df['ATA_LT'] < 11), 'group_time'] = 'morning'
    df.loc[(df['ATA_LT'] >= 11) & (df['ATA_LT'] < 18), 'group_time'] = 'afternoon'
    df.loc[(df['ATA_LT'] >= 18) & (df['ATA_LT'] <= 23), 'group_time'] = 'dinner'
    df.loc[(df['group_time'] == '-'), 'group_time'] = 'Na'
    return df['group_time']

def group_built(df):
    df['group_built'] = '-' 
    df.loc[(df['BUILT'] < 10), 'group_built'] = 'age_1'
    df.loc[(df['BUILT'] >= 10) & (df['BUILT'] < 20), 'group_built'] = 'age_2'
    df.loc[(df['BUILT'] >= 20) & (df['BUILT'] < 30), 'group_built'] = 'age_3'
    df.loc[(df['BUILT'] >= 40) & (df['BUILT'] < 50), 'group_built'] = 'age_4'
    df.loc[(df['BUILT'] >= 50) & (df['BUILT'] < 60), 'group_built'] = 'age_5'
    df.loc[(df['BUILT'] >= 60) & (df['BUILT'] < 70), 'group_built'] = 'age_6'
    df.loc[(df['BUILT'] >= 70) & (df['BUILT'] <= 80), 'group_built'] = 'age_7'
    df.loc[(df['group_built'] == '-'), 'group_built'] = 'Na'
    return df['group_built']

def log_DIST(df):
    df['log_DIST'] = 0
    df['log_DIST'] = np.log1p(df['DIST'])
    return df['log_DIST']

def DIST_LENGTH(df):
    df['DIST_LENGTH'] = 0
    df['DIST_LENGTH'] = df['DIST']/df['LENGTH']
    return df['DIST_LENGTH']

In [ ]:
def make_dataset(train, test):
    
    start = datetime.now()
    print('Start time: ', start)
    
    train['year_month'] = train['year'].astype(str) + '_' + train['month'].astype(str)
    train['group_covid'] = group_covid(train)
    train['group_season'] = group_season(train)
    train['group_day'] = group_day(train)
    train['group_time'] = group_time(train)
    train['group_built'] = group_built(train)
    train['log_DIST'] = log_DIST(train)
    train['DIST_LENGTH'] = DIST_LENGTH(train)
    
    print('Train dataset success !')
    
    test['year_month'] = test['year'].astype(str) + '_' + test['month'].astype(str)
    test['group_covid'] = group_covid(test)
    test['group_season'] = group_season(test)
    test['group_day'] = group_day(test)
    test['group_time'] = group_time(test)
    test['group_built'] = group_built(test)
    test['log_DIST'] = log_DIST(test)
    test['DIST_LENGTH'] = DIST_LENGTH(test)
    
    print('Test dataset success !')

    str_col = ['year_month' , 'group_covid', 'group_season', 'group_day', 'group_time', 'group_built', 'ARI_INFO']
    
    for i in str_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]):
            if label not in le.classes_:
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i])
    
    X = train.drop(    
        ['SAMPLE_ID' ,'ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'date', 'FLAG', 'CI_HOUR'], axis=1
    )

    y = train['CI_HOUR']

    test = test.drop(
        ['SAMPLE_ID', 'ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'date', 'FLAG'], axis=1
    )

    End = datetime.now()
    print(f'End time: {End}')
    print('Play time: ', End - start)
    
    return X, y, test

In [ ]:
X, y, X_test = make_dataset(train, test)

In [ ]:
#Time Cycling Transform 
##시간
X['sin_hour'] = np.sin(2 * np.pi * X['hour']/23.0)
X['cos_hour'] = np.cos(2 * np.pi * X['hour']/23.0)
X_test['sin_hour'] = np.sin(2 * np.pi * X_test['hour']/23.0)
X_test['cos_hour'] = np.cos(2 * np.pi * X_test['hour']/23.0)

##분
X['sin_minute'] = np.sin(2 * np.pi * X['minute']/59.0)
X['cos_minute'] = np.cos(2 * np.pi * X['minute']/59.0)
X_test['sin_minute'] = np.sin(2 * np.pi * X_test['minute']/59.0)
X_test['cos_minute'] = np.cos(2 * np.pi * X_test['minute']/59.0)

##날짜
X['sin_date'] = -np.sin(2 * np.pi * (X['month']+X['day']/31)/12)
X['cos_date'] = -np.sin(2 * np.pi * (X['month']+X['day']/31)/12)
X_test['sin_date'] = -np.sin(2 * np.pi * (X_test['month']+X_test['day']/31)/12)
X_test['cos_date'] = -np.sin(2 * np.pi * (X_test['month']+X_test['day']/31)/12)

##월
X['sin_month'] = -np.sin(2 * np.pi * X['month']/12.0)
X['cos_month'] = -np.cos(2 * np.pi * X['month']/12.0)
X_test['sin_month'] = -np.sin(2 * np.pi * X_test['month']/12.0)
X_test['cos_month'] = -np.cos(2 * np.pi * X_test['month']/12.0)

## 4. Model : LGBM & XGBOOST
- optuna 사용 시, 학습데이터에 대해 Overfitting이 발생할 가능성이 있어 직접 파라미터들을 변경해가며 튜닝을 진행하였습니다

In [ ]:
is_holdout = False
iterations = 50000
patience = 500

In [ ]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

models_lgb = []
models_xgb = []
mae_scores = []
n_split_list = [10]

for split in n_split_list:
    fold_idx = 1
    cv = KFold(n_splits=split, shuffle=True, random_state=42)
    for train_index, valid_index in cv.split(X):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        Y_train, Y_valid = y[train_index], y[valid_index]
        print("="*50)

        model_lgb = LGBMRegressor(boosting_type='gbdt',
                            objective='regression_l1', 
                            n_estimators=iterations,
                            max_depth=12,
                            learning_rate=0.07,
                            colsample_bytree=0.9,
                            subsample=1.0,
                            min_child_weight=60,
                            num_leaves=512,
                            n_jobs=-1,
                            random_state=42)   
                             
    
        model_lgb.fit(X_train, Y_train, 
                  eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                  early_stopping_rounds=patience, 
                  verbose=100)
        
        model_xgb = XGBRegressor(n_estimators = iterations, eta = 0.15, min_child_weight = 60, 
                       max_depth = 10, colsample_bytree = 0.8,
                       subsample = 1.0, seed = 42,
                       objective = 'reg:absoluteerror',
                       eval_metric = 'mae',
                       tree_method="gpu_hist")
    
        model_xgb.fit(X_train, Y_train, 
                  eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                  early_stopping_rounds=patience, verbose=100)
        
        pred_lgb = model_lgb.predict(X_valid)
        pred_xgb = model_xgb.predict(X_valid)
        pred = pred_lgb*0.8 + pred_xgb*0.2
        score = mean_absolute_error(Y_valid,pred)
        print(fold_idx,"Fold Validation MAE score :", score)
        mae_scores.append(score)
        models_lgb.append(model_lgb)
        models_xgb.append(model_xgb)
        fold_idx += 1
        if is_holdout:
            break 
print("Validation : MAE scores for each fold:", mae_scores)
print("Validation : MAE:", np.mean(mae_scores))

In [ ]:
#LGBM feature importance
for i in range(10):
    predictors = X.columns
    tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': models_lgb[i].feature_importances_})
    tmp = tmp.sort_values(by='Feature importance',ascending=False)
    plt.figure(figsize = (7,4))
    plt.title('Features importance',fontsize=7)
    s = sns.barplot(x='Feature',y='Feature importance',data=tmp)
    s.set_xticklabels(s.get_xticklabels(),rotation=90, size=10)
    plt.show()

In [ ]:
#XGBOOST feature importance
for i in range(10):
    predictors = X.columns
    tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': models_xgb[i].feature_importances_})
    tmp = tmp.sort_values(by='Feature importance',ascending=False)
    plt.figure(figsize = (7,4))
    plt.title('Features importance',fontsize=7)
    s = sns.barplot(x='Feature',y='Feature importance',data=tmp)
    s.set_xticklabels(s.get_xticklabels(),rotation=90, size=10)
    plt.show()

## 5. test inference

In [ ]:
preds = []
for i in range(10):
    pred_lgb = models_lgb[i].predict(X_test)
    pred_xgb = models_xgb[i].predict(X_test)
    pred = pred_lgb*0.8 + pred_xgb*0.2
    preds.append(pred)

preds = np.mean(preds , axis = 0)

In [ ]:
#예측값 할당
submission = pd.read_csv('./data/sample_submission.csv')
submission['CI_HOUR'] = preds

#DIST가 0인 Target 값 후처리
index_0 = test[test['DIST'] == 0].index
submission.loc[index_0, 'CI_HOUR'] = 0.0

#음수값 후처리
submission.loc[submission['CI_HOUR'] < 0, 'CI_HOUR'] = 0.0

submission.to_csv('./submissions/final_sub_woo.csv', index = False)